In [1]:
import pandas as pd
import numpy as np
import os
import json

pd.set_option('display.max_columns', 500)

In [2]:
'''
-read in the 2.Indices_Target.csv file
-filter on the NA-s in the snapshot_target column
-get the unique target nums
'''

filename='2.Indices_Target.csv'
df_target=pd.read_csv(f'C:/Users/511232/Desktop/Scorecard_coxcomb/{filename}', dtype='str')

#filter out the dataframe where snapshot_target is NA
df_na=df_target[df_target['snapshot_target'].isna()].copy()
target_num_NA=set(df_na['Target_number'])

In [3]:
'''
Code	Color
0	    Grey
1	    Red
2	    Yellow
3	    Green

-the scorecard.xlsx file, filter on color ('Color Scheme'=0) & ('Gap'=blank) & ('target' in target_num_NA)
-groupby ['geoareaname', 'target'] and apply a function that will take the average of 'Trend'
-and standardize the result of the average to be between [-10;+10]
formula=N= -10 + [(A+100)/10]
'''

filename='scorecard.xlsx'
#read in the scorecard.xlsx
df=pd.read_excel('C:/Users/511232/Desktop/Scorecard_coxcomb/scorecard.xlsx', dtype=str)
df['Trend'] = df['Trend'].astype(float)

#flag as 1 if (df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA)) condition meets
cond=(df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA))
df.loc[cond,'flag']=1

#filter color scheme=0 & Gap=blank
df_filtered=df[(df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA))].copy()


def process(d):
    '''will calculate average trend and standardize the value'''
    trend_avg=d['Trend'].mean()
    trend_avg_standardized=-10+((trend_avg+100)/10)
    return(trend_avg_standardized)

#apply the function on the df_filtered
standardized_df=pd.DataFrame(df_filtered.groupby(['geoareaname','target'],group_keys=False).apply(process))
standardized_df.reset_index(drop=False, inplace=True)
standardized_df.columns=['geoareaname','target','standardized_value']
#add flag=1
standardized_df['flag']=1

#merge the standardized_df with df
result=pd.merge(df,standardized_df, how='left', on=['geoareaname','target', 'flag'])

In [9]:
'''
-generate coxcomb color using the following benchmarks:
if N is negative --> red, takes the value of 1
if N is postive but less or equal to 5.33 --> yellow, takes the value of 2
if N is greater than 5.33--> green, takes the value of 3"
'''
condition=[result['standardized_value']<0,(result['standardized_value']>0) & (result['standardized_value']<=5.33), result['standardized_value']>5.33]
colors=[1,2,3]
result['calculated_color']=np.select(condition,colors, default=np.nan)

result.to_excel('processed_file.xlsx',index=False)

### discover the cutoff point for Yemen

In [109]:
df=pd.DataFrame({'year':range(2000,2024), 'val':[100,85,75,70,75,65,58,55,80,60,50,80,90,100,120,130,135,125,140,145,150,155,158,160]})
df

,year,val
0,2000,100
1,2001,85
2,2002,75
3,2003,70
4,2004,75
5,2005,65
6,2006,58
7,2007,55
8,2008,80
9,2009,60


In [112]:
#get the percentages of False and True
b=df['val'].pct_change()>0
b.value_counts(normalize=True)*100

True     58.333333
False    41.666667
Name: val, dtype: float64

In [174]:
#get the indices of one of the chunks of False or True
ix=b[b].index
print(ix)

idx=[i for i in range(len(ix)-1) if ix[i+1]-ix[i]>=3]

if idx:
    print(f'the cutoff indeex is :{ix[idx[-1]]}')
else:
    print('returned empty list')

Int64Index([4, 8, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23], dtype='int64')
the cutoff indeex is :8


In [2]:
os.chdir('C:/Users/511232/Desktop/Scorecard_coxcomb')
df=pd.read_excel('data Yemen.xlsx')

In [15]:
def report_on_series(d):
    global groupby_vars
    global s
    global result_per_serie
    global series_counter
    global series_counter_2dp
    global series_counter_3plusdp
    global series_count

    #get the categories labels
    k=list(d.name)
    k=[str(i) for i in k]

    #report on the columns/names used in groupby
    c=groupby_vars
    categ_name=dict(zip(c,k))

    #update a dictionary that will track number of datapoints for each groupby dataframes
    series_count_key=f'{len(d)} datapoints'
    if series_count_key in series_count.keys():
        series_count[series_count_key]+=1
    else:
        series_count[series_count_key]=1

    
    if len(d)==0:
        result_per_serie.append({'series':s, 'categories':categ_name, 'number of data_points':len(d)})
    elif (len(d)>0) & (len(d)<=2):
        result_per_serie.append({'series':s, 'categories':categ_name, 'number of data_points':len(d)})
    else:
        result_per_serie.append({'series':s, 'categories':categ_name, 'number of data_points':len(d)})

    

In [16]:
#remove these from groupby vars
list_to_exclude=['BasePeriod', 'TimeCoverage', 'Time_Detail','LowerBound', 'UpperBound', 'FootNote','GeoInfoUrl', 'Source','Reporting Type', 'Sampling Stations']

#get unique SERIES
unique_series=list(df['SeriesCode'].unique())

result_per_serie=[] #initialize for each unique series
series_count={}     #it will count indicators count


for s in unique_series:
    #get the dataframe with SERIES == s
    df_s=df[df['SeriesCode']==s].copy()
    #get rid of null columns
    df_s.dropna(how='all',axis=1, inplace=True)
    #get the groupby vars which is the column names to groupby with
    cols=df_s.columns
    cols=set(cols).difference(set(list_to_exclude))
    groupby_vars=list(cols.difference({'TimePeriod', 'Value'}))
    
    # apply Sen slope on the dataframe
    df_s.groupby(groupby_vars, as_index=True).apply(report_on_series)



#add the counter result to result_per_serie
result_per_serie.append(series_count)

#write it to a json file
with open('report.json', 'w') as outfile:
    json.dump(result_per_serie,outfile, indent=1)